# Debugging Assignment — Employee Attrition Prediction (BROKEN VERSION)

This notebook contains an **intentionally broken** machine learning pipeline for predicting employee attrition.

Your task (as a candidate) will be to:
- Identify the issues
- Fix them
- Retrain and properly evaluate the model

> ⚠️ Note: This version is deliberately incorrect. Do **not** use it as a template in production.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Load dataset
df = pd.read_csv("../csv/debug.csv")
print(df.head())


   age  gender      education department   job_role  monthly_income  \
0   50  Female  Post-Graduate         IT       Lead          102565   
1   36  Female            PhD      Sales       Lead           49402   
2   29  Female       Graduate         HR  Executive           24263   
3   42    Male       Graduate         HR  Executive          116523   
4   40  Female            PhD         HR    Manager           66828   

   years_at_company  promotions overtime  performance_rating  attrition  
0                 9           0       No                   1          0  
1                 2           0      Yes                   2          0  
2                 7           1       No                   1          0  
3                 7           3      Yes                   4          0  
4                 1           1      Yes                   3          0  


In [4]:

# Basic info
print(df.shape)
print(df.dtypes)
print(df.isna().sum())
df.describe()


(300, 12)
age                    int64
gender                object
education             object
department            object
job_role              object
monthly_income         int64
years_at_company       int64
promotions             int64
overtime              object
performance_rating     int64
attrition              int64
attrition_copy         int64
dtype: object
age                   0
gender                0
education             0
department            0
job_role              0
monthly_income        0
years_at_company      0
promotions            0
overtime              0
performance_rating    0
attrition             0
attrition_copy        0
dtype: int64


,age,monthly_income,years_at_company,promotions,performance_rating,attrition,attrition_copy
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,41.186667,69834.483333,7.243333,2.073333,2.500000,0.230000,0.230000
std,11.275739,28201.559946,4.454616,1.421745,1.172426,0.421536,0.421536
min,22.000000,20060.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,30.000000,46111.750000,3.000000,1.000000,1.000000,0.000000,0.000000
50%,43.000000,69012.000000,8.000000,2.000000,2.500000,0.000000,0.000000
75%,51.000000,94733.000000,11.000000,3.000000,4.000000,0.000000,0.000000
max,59.000000,119804.000000,14.000000,4.000000,4.000000,1.000000,1.000000


In [12]:

# --- Feature Engineering (BROKEN ON PURPOSE) ---

# 1) Create a feature that directly uses the target (leakage disguised as a helper feature)

df["attrition_copy"] = df["attrition"]  # <-- target copied into features


# 2) Use 'target_leakage_feature' as-is assuming it's a good predictor

# (We don't check how this was created.)


# 3) Fill all missing values with 0 without analysis

df = df.fillna(0)

df.head()


,age,gender,education,department,job_role,monthly_income,years_at_company,promotions,overtime,performance_rating,attrition,attrition_copy
0,50,Female,Post-Graduate,IT,Lead,102565,9,0,No,1,0,0
1,36,Female,PhD,Sales,Lead,49402,2,0,Yes,2,0,0
2,29,Female,Graduate,HR,Executive,24263,7,1,No,1,0,0
3,42,Male,Graduate,HR,Executive,116523,7,3,Yes,4,0,0
4,40,Female,PhD,HR,Manager,66828,1,1,Yes,3,0,0


In [14]:

# --- Create features/target (order BROKEN) ---
categorical_cols = ['gender', 'education', 'department', 'job_role', 'overtime']

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

X = df.drop("attrition", axis=1)

y = df["attrition"]


# Scale ALL data before splitting (data leakage)

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)


# Split AFTER scaling (this uses information from the full dataset)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)



In [18]:
X_train

array([[-0.01658238,  0.33012523,  0.39500642, ...,  1.40367269,
        -0.70181003,  1.09077084],
       [ 0.42758843,  1.65545425,  1.06959043, ..., -0.71241679,
        -0.70181003,  1.09077084],
       [ 0.24992011, -1.4320431 , -0.50443894, ...,  1.40367269,
        -0.70181003,  1.09077084],
       ...,
       [ 0.24992011,  0.57704829, -0.72930027, ..., -0.71241679,
        -0.70181003, -0.91678285],
       [-1.08259231, -1.00568169,  0.84472909, ...,  1.40367269,
        -0.70181003, -0.91678285],
       [ 1.13826172, -1.39297299,  0.17014508, ..., -0.71241679,
         1.42488702,  1.09077084]], shape=(240, 15))

In [19]:

# --- Train RandomForest with almost no control ---

model = RandomForestClassifier(n_estimators=500)

model.fit(X_train, y_train)


# Evaluate using Accuracy only

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print("Test Accuracy:", acc)


print("\nClassification Report:")

print(classification_report(y_test, y_pred))



Test Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        19

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [20]:
# --- Misused cross-validation (BROKEN) ---

# Run cross-validation only on the TEST set (incorrect!)

cv_scores = cross_val_score(model, X_test, y_test, cv=5)

print("Cross-validation scores on test set:", cv_scores)

print("Mean CV score:", cv_scores.mean())

Cross-validation scores on test set: [1. 1. 1. 1. 1.]
Mean CV score: 1.0
